In [27]:
from imutils.perspective import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import cv2
import imutils


In [28]:
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged

In [67]:
def document_scanner(image_path):
    # 1. Read the image
    image = cv2.imread(image_path)

    # To make edge detection step more accurate, we resize our scanned image to a height of 500 pixels
    # Ratio will allow us to perform the scan on the original image rather than the resized image.
    ratio = image.shape[0] / 500.0
    orig = image.copy()
    image = imutils.resize(image, height = 500)
    
    # 2. Edge Detection
    
    # Convert the image to grayscale, blur it, and find edges (canny edged detecter is used)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Smooth the image with a Gaussian filter to reduce noise and unwanted details and textures. 
    # which will help edge detection process
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    #edged = cv2.Canny(gray, 75, 200) 
    edged = auto_canny(gray)
#     plt.imshow(edged)
#     plt.show()
#     plt.imshow(edged2)
#     plt.show()
    # 3. Finding Contours
    
    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True) # Because we want to examine only large contours
    #print(len(cnts))
    # we’ll assume that the largest contour in the image with exactly four points is our piece of paper to be scanned.
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        if len(approx) == 4:
            screenCnt = approx
            break

    # Find contours of paper
    cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)

    
    # 4. Apply the four point perspective transform to obtain a bird eye view of the document
    warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)

    # Convert the warped image to grayscale, then threshold it
    # To give it that 'black and white' paper effect
    warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    T = threshold_local(warped, 11, offset = 10, method = "gaussian")
    warped = (warped > T).astype("uint8") * 255
    return orig, warped

In [69]:
a, b = document_scanner("images/page.jpg")

In [70]:
# Display the results.
cv2.namedWindow("orig", cv2.WINDOW_NORMAL)  
cv2.namedWindow("scanned", cv2.WINDOW_NORMAL)   
cv2.imshow("orig", a)
cv2.imshow("scanned", b)
cv2.waitKey(0)
cv2.destroyAllWindows()